Notes on complexity
  - n means 'number of elements'
  - c means 'maximum number of columns for elements being sorted'
    - eg. 999 has 3 columns in base-10. In base-2 this is a 16 bit number, so we would say it has 2 columns if we are considering 1 byte at a time, or 16 columns if we are considering 1 bit at a time.
  - k means 'number of bits per column'
    - eg. in base-10 every column has 1 bit, but in base-2, char has 8 bits, uint_16 has 16 bits, etc..

  - Counting Sort: O(n+ck)
  - Radix Sort: O(c(n+k))
  - Merge Sort: O(n*log_2(n))

    These O complexities are nice for getting a feel for the performance of an algorithm, but they don't tell the whole story.

    Radix sort is basically the same thing as counting sort.
    In both algorithms, you count the number of times that each value appears, and then you order the items based on the count.
    The problem with counting sort for some datasets is the number of discrete values that could appear in the dataset.
    For example, consider a list of words where each word has 10 characters. The maximum number of discrete values is 26^10. That's... hmm. A lot.
    However, if you line the words up - 1 word per row - and then sort each column, you're dealing with, at maximum, 26 discrete values per column. That's managable. Thats radix.

    Now, lets assume the human architecture.
    How long does it take you to remember wether j or k comes first in the alphabet?
    Then, how long does it take you to write it down, once you remember it?
    How much graph paper do you need to perform the task, and how long will it take you to retrieve the graph paper?
    Do you know how much graph paper you will need when you start, or will you need to keep fetching more as you go?

    In rsort_lsb_v1 I have opted to use a lot of memory on the stack to avoid repeating tasks. Is this faster than other methods of performing radix? I don't know until I test it against other methods.
    Another method, which seems to be widely accepted, is to itterate through each column twice: the first time you count, and the second time you move the items (or pointers to the items). This strategy uses several times less memory than rsort_lsb_v1, but by my estimation, it takes more time to complete.
    By the way, using several times more memory is fine for plenty of applications. For sorting large word lists (i.e. for search engines), we're talking about a few megabytes per thread.
    Another way to use less memory is to allocate the memory as-needed on the heap, but allocating the memory takes time, and then accessing the memory on the heap takes more time (and a less predictable amount of time) compared to if it were on the stack.
    
    So, eventually, I think I will write all three methods (and perhaps more) and compare them all.
    The best method gets multithreaded - that's the real power of radix sort. It can definately be multithreaded efficiently. All you have to do is split each column up into sub-columns, then schedule the threads to do work on the subcolumns. Synchronizing counts needs a mutex, then writing to memory can be done in parallel.


Additional notes:
  - The MSB variant of radix sort would be fantastic for text-based searches. Think about it.
    - edit: well, I thought about it a little bit more. Doing rsort in the true forward configuration is next to impossible for any word size over 5 (26^8 = hmm. that's a lot.)
    - So what do we do instead for single-character additions to a word? A binary comparison across the column. Preferably, multithreaded. This basically looks the same as performing a column of radix, except (depending on the application) we discard all the values that don't match. Well, that isn't a sort anymore, is it? That's a search.

  - Why isn't arrmove in the c standard library? Haha.

  - bsearch is fantastic, but only if you have a sorted array.
    - edit: well, see 'additional note #1.' Depending on the application, you might just roll your search requirements directly into radix and do it all in one function call.
    - note to self: this functionality could be acheived using essentially the same rsort functions with a more robust tokenizer function.

If we take the O complexities at face-value, we see the following characteristics when considering text-based datasets:

In [3]:
import sympy as sp
from math import ceil

n = sp.symbols('n')

merge_sort = n * sp.log(n, 2)

for c in range(1, 50):
    radix_sort = c * (n + 8)
    eqn2 = sp.Eq(merge_sort, radix_sort)
    sol2 = sp.solve(eqn2, n)
    for solution in sol2:
        print(f"When there are {c} column(s) in the dataset, it's a good idea to use radix sort instead of merge sort if there are more than {ceil(solution.evalf())} rows in the dataset.")



When there are 1 column(s) in the dataset, it's a good idea to use radix sort instead of merge sort if there are more than 6 rows in the dataset.
When there are 2 column(s) in the dataset, it's a good idea to use radix sort instead of merge sort if there are more than 11 rows in the dataset.
When there are 3 column(s) in the dataset, it's a good idea to use radix sort instead of merge sort if there are more than 20 rows in the dataset.
When there are 4 column(s) in the dataset, it's a good idea to use radix sort instead of merge sort if there are more than 32 rows in the dataset.
When there are 5 column(s) in the dataset, it's a good idea to use radix sort instead of merge sort if there are more than 54 rows in the dataset.
When there are 6 column(s) in the dataset, it's a good idea to use radix sort instead of merge sort if there are more than 92 rows in the dataset.
When there are 7 column(s) in the dataset, it's a good idea to use radix sort instead of merge sort if there are more t

O complexity doesn't tell the whole story. Theres a lot more going on under the hood with these algorithms.

However, this output is an indicator that radix is a top-contender sorting algorithm for real-time text-based searches.

Often, the input comes in 1 character (1 column) at a time.

You want to be able to sort (resort) the dataset(s), then search them, in a few microseconds each time that happens.